In [2]:
import os
import pandas as pd
import numpy as np
import time
import torch
from torch.utils.data import DataLoader, TensorDataset, random_split
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning import Trainer, seed_everything
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.feature_selection import mutual_info_classif, chi2
from sklearn.linear_model import LassoCV
import matplotlib.pyplot as plt
from pytorch_lightning.loggers import TensorBoardLogger
import seaborn as sns
import os
import sys
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from torch_explain.models.explainer import Explainer
from torch_explain.logic.metrics import formula_consistency
# from load_datasets import load_mimic
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTEN
from imblearn.combine import SMOTEENN
from torch.nn.functional import one_hot
from func_timeout import func_set_timeout, func_timeout, FunctionTimedOut
from datetime import date

In [3]:
files = os.listdir("./categorisedData/")


datasets = {file : pd.read_csv("./categorisedData/" + file) for file in files}


print(files)

results_dict = {}

['breastCancer.csv', 'clusteredData.csv', 'expertLabelledData.csv', 'metricExtractedData.csv', 'staticData.csv']


In [4]:
@func_set_timeout(90)
def explain_with_timeout(model, val_data, train_data, test_data, topk_expl, concepts):

    return model.explain_class(val_dataloaders=val_data, train_dataloaders=train_data, test_dataloaders=test_data, topk_explanations=topk_expl, concept_names=concepts, max_minterm_complexity=2)

In [6]:

np.set_printoptions(threshold=sys.maxsize)

# print(os.listdir("."))


for file in files:

    # file = "breastCancer.csv"


    print(f"Training {file}\n")

    data = datasets[file]

    if "PatientID" in data.columns:
        data = data.drop(columns=["PatientID"])


    targetName = "Mortality14Days"

    targetSeries = data[targetName]
    print(data[targetName].value_counts())
    data = data.drop(columns=[targetName])

    dataTensor = torch.FloatTensor(data.to_numpy())
    targetTensor = one_hot(torch.tensor(targetSeries.values).to(torch.long)).to(torch.float)

    # x, y, concept_names = load_mimic()
    # base_dir="./LEN_test/pytorch_explain/experiments/elens/data"


    dataset = TensorDataset(dataTensor, targetTensor)

    total_loader = DataLoader(dataset)


    train_size = int(len(dataset) * 0.5)
    val_size = (len(dataset) - train_size) // 2
    test_size = len(dataset) - train_size - val_size
    train_data, val_data, test_data = random_split(dataset, [train_size, val_size, test_size])
    train_loader = DataLoader(train_data, batch_size=train_size, shuffle=True)
    val_loader = DataLoader(val_data, batch_size=val_size)
    test_loader = DataLoader(test_data, batch_size=test_size)
    n_concepts = next(iter(train_loader))[0].shape[1]
    n_classes = 2
    # print("feature names: ", concept_names)
    print("features:", n_concepts)
    # print(n_classes)

    # %% md

    ## 5-fold cross-validation with explainer network

    # %%

    seed_everything(42)

    base_dir = f'./runs'
    # os.makedirs(base_dir, exist_ok=True)

    n_splits = 5
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
    
    results_list = []
    feature_selection = []
    explanations = {i: [] for i in range(n_classes)}

    explanations_list = []
    splitResults_list = []
    scores_list = []


    x = dataTensor
    y = targetTensor

    for split, (trainval_index, test_index) in enumerate(skf.split(x.cpu().detach().numpy(),
                                                                y.argmax(dim=1).cpu().detach().numpy())):
        print(f'Split [{split + 1}/{n_splits}]')

        # print(x.shape)

        x_trainval, x_test = torch.FloatTensor(x[trainval_index]), torch.FloatTensor(x[test_index])
        y_trainval, y_test = torch.FloatTensor(y[trainval_index]), torch.FloatTensor(y[test_index])
        x_train, x_val, y_train, y_val = train_test_split(x_trainval, y_trainval, test_size=0.2, random_state=42)
        print(f'{len(y_train)}/{len(y_val)}/{len(y_test)}')

        print(pd.Series(np.argmax(y_train.numpy(), axis=1)).value_counts().values)

        clf = SMOTEN(random_state=0)

        x_train, y_train = clf.fit_resample(x_train.numpy(), np.argmax(y_train.numpy(), axis=1))

        x_train = torch.FloatTensor(x_train)
        y_train = one_hot(torch.tensor(y_train).to(torch.long)).to(torch.float)

        print(pd.Series(np.argmax(y_train.numpy(), axis=1)).value_counts().values)


        train_data = TensorDataset(x_train, y_train)
        val_data = TensorDataset(x_val, y_val)
        test_data = TensorDataset(x_test, y_test)
        train_loader = DataLoader(train_data, batch_size=train_size, shuffle=True)
        val_loader = DataLoader(val_data, batch_size=val_size)
        test_loader = DataLoader(test_data, batch_size=test_size)

        checkpoint_callback = ModelCheckpoint(dirpath=base_dir, monitor='f1_val', mode='max', save_top_k=1)

        logger = TensorBoardLogger("./runs/splits/", name=file)

        trainer = Trainer(max_epochs=400, gpus=1, auto_lr_find=True, deterministic=True,
                        check_val_every_n_epoch=1, default_root_dir=base_dir,
                        weights_save_path=base_dir, callbacks=[checkpoint_callback],
                        logger=logger, enable_progress_bar=False, gradient_clip_val=0.5)


        model = Explainer(n_concepts=n_concepts, n_classes=n_classes, l1=1e-3, lr=0.01,
                        explainer_hidden=[20], temperature=0.7)

        start = time.time()
        trainer.fit(model, train_loader, val_loader)
        # print(f"Gamma: {model.model[0].concept_mask}")
        model.freeze()

        # Precision, Recall, F1
        y_pred = torch.argmax(model(x_test), axis=1)
        y_test_argmax = torch.argmax(y_test, axis=1)

        scores = [f1_score(y_test_argmax.numpy(), y_pred.numpy(), average='macro'), 
                recall_score(y_test_argmax.numpy(), y_pred.numpy(), average='macro'), 
                precision_score(y_test_argmax.numpy(), y_pred.numpy(), average='macro')]

        print(f"Before loading best: {scores}")

        # scores_list.append(scores)
    
        model = model.load_from_checkpoint(checkpoint_callback.best_model_path)

        

        # Precision, Recall, F1

        scores = [f1_score(y_test_argmax.numpy(), y_pred.numpy(), average='macro'), 
                recall_score(y_test_argmax.numpy(), y_pred.numpy(), average='macro'), 
                precision_score(y_test_argmax.numpy(), y_pred.numpy(), average='macro')]

        print(f"{file} split {split+1} scores: {scores}")

        print("\nTesting...\n")
        model_results = trainer.test(model, dataloaders=test_loader)

        scores_list.append(scores)


        print("\nExplaining\n")

        try:

            results, f = explain_with_timeout(model, val_data=val_loader, train_data=train_loader, test_data=test_loader,
                                        topk_expl=10,
                                        concepts=data.columns)

        except FunctionTimedOut:
            print("Explanation timed out, skipping...")
            # explanations_list.append(None)
            # results_list.append(None)
            continue

        end = time.time()
        # explanations_list.append(f)

        print(f"Explaining time: {end - start}")
        results['model_accuracy'] = model_results[0]['test_acc_epoch']
        results['extraction_time'] = end - start

        results_list.append(results)
        extracted_concepts = []
        all_concepts = model.model[0].concept_mask[0] > 0.5
        common_concepts = model.model[0].concept_mask[0] > 0.5
        for j in range(n_classes):
            # print(f[j]['explanation'])
            n_used_concepts = sum(model.model[0].concept_mask[j] > 0.5)
            print(f"Number of features that impact on target {j}: {n_used_concepts}")
            print(f"Explanation for target {j}: {f[j]['explanation']}")
            print(f"Explanation accuracy: {f[j]['explanation_accuracy']}")
            explanations[j].append(f[j]['explanation'])
            extracted_concepts.append(n_used_concepts)
            all_concepts += model.model[0].concept_mask[j] > 0.5
            common_concepts *= model.model[0].concept_mask[j] > 0.5

        explanations_list.append(explanations)

        results['extracted_concepts'] = np.mean(extracted_concepts)
        results['common_concepts_ratio'] = sum(common_concepts) / sum(all_concepts)

        

        # prec_rec = precision_recall(y_pred, y_test_argmax, num_classes = n_classes)

        # print(prec_rec)

        # compare against standard feature selection
        i_mutual_info = mutual_info_classif(x_trainval, y_trainval[:, 1])
        i_chi2 = chi2(x_trainval, y_trainval[:, 1])[0]
        i_chi2[np.isnan(i_chi2)] = 0
        lasso = LassoCV(cv=5, random_state=0).fit(x_trainval, y_trainval[:, 1])
        i_lasso = np.abs(lasso.coef_)
        i_mu = model.model[0].concept_mask[1]
        # print(model.model[0].concept_mask)
        df = pd.DataFrame(np.hstack([
            i_mu.numpy(),
            # i_mutual_info / np.max(i_mutual_info),
            # i_chi2 / np.max(i_chi2),
            # i_lasso / np.max(i_lasso),
        ]).T, columns=['feature importance'])
        df['method'] = 'explainer'
        # df.iloc[90:, 1] = 'MI'
        # df.iloc[180:, 1] = 'CHI2'
        # df.iloc[270:, 1] = 'Lasso'
        df['feature'] = np.hstack([np.arange(0, n_concepts)])
        feature_selection.append(df)

        splitResults = [results['model_accuracy'], results['extraction_time'], *scores, f]

        splitResults_list.append(splitResults)


    results_dict[file] = splitResults_list


Global seed set to 42


Training breastCancer.csv

0    458
1    241
Name: Mortality14Days, dtype: int64
features: 89
Split [1/5]
447/112/140
[298 149]


c:\Users\benma\anaconda3\lib\site-packages\pytorch_lightning\trainer\connectors\callback_connector.py:57: LightningDeprecationWarning: Setting `Trainer(weights_save_path=)` has been deprecated in v1.6 and will be removed in v1.8. Please pass ``dirpath`` directly to the `ModelCheckpoint` callback
  rank_zero_deprecation(
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
c:\Users\benma\anaconda3\lib\site-packages\pytorch_lightning\utilities\parsing.py:261: UserWarning: Attribute 'loss' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss'])`.
  rank_zero_warn(
c:\Users\benma\anaconda3\lib\site-packages\pytorch_lightning\callbacks\model_checkpoint.py:611: UserWarning: Checkpoint directory C:\Users\benma\OneDrive\Kings\Modules\Term 2\Individual Project\LEN Individual Project\Notebooks\runs exists a

[298 298]


c:\Users\benma\anaconda3\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:240: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
c:\Users\benma\anaconda3\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:240: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
c:\Users\benma\anaconda3\lib\site-packages\pytorch_lightning\trainer\trainer.py:1933: PossibleUserWarning: The number of training batches (2) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a 

...

[2931 2931]


c:\Users\benma\anaconda3\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:240: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
c:\Users\benma\anaconda3\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:240: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
c:\Users\benma\anaconda3\lib\site-packages\pytorch_lightning\trainer\trainer.py:1933: PossibleUserWarning: The number of training batches (3) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a 

Before loading best: [0.5389053186557233, 0.5576515540714102, 0.5383500200240289]
staticData.csv split 5 scores: [0.5234960532911386, 0.5234234523503725, 0.5235712266246617]

Testing...

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      f1_test_epoch         0.5234960317611694
     test_acc_epoch          0.786121666431427
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Explaining



c:\Users\benma\anaconda3\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:240: PossibleUserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Explaining time: 46.974565505981445
Number of features that impact on target 0: 13
Explanation for target 0: gender_male | ~coagulation_medium
Explanation accuracy: 0.48539133738601825
Number of features that impact on target 1: 13
Explanation for target 1: None
Explanation accuracy: 0


In [7]:
y_pred = torch.argmax(model(x_train), axis=1)

y = torch.argmax(y_train, axis=1)

print("train f1:" , f1_score(y, y_pred, average='macro'))

train f1: 0.8449716428842396


In [8]:
y_pred = torch.argmax(model(x_test), axis=1)

y = torch.argmax(y_test, axis=1)

print("test f1:", f1_score(y, y_pred, average='macro'))

test f1: 0.5234960532911386


In [24]:
kFoldMeans = []

bestExplanationsDict = {f:[0,0] for f in results_dict.keys()}

# print(bestExplanationsDict)

for x in results_dict:

    cols = ['file', 'model_accuracy', 'extraction_time', 'f1', 'recall', 'precision']

    # cols.extend(['model_accuracy', 'extraction_time', 'f1', 'recall', 'precision'])

    for idx, d in enumerate(results_dict[x][0][5]):
        cols.extend([str(x) + "_" + str(idx) for x in list(d)[1:]])

    # print(cols)

    rows = []

    for split in results_dict[x]:


        if split[5][1]['explanation_accuracy'] > bestExplanationsDict[x][0]:
            bestExplanationsDict[x] = [split[2], split[5]]

        row = [x]

        row.extend(split[:5])


        for d in split[5]:

            row.extend(list(d.values())[1:])


        

        rows.append(row)


    df = pd.DataFrame(columns=cols, data=rows)

    df = df.set_index('file')

    combinedCols = list(df.describe().columns)

    # print(combinedCols)

    row = [x]
    row.extend(np.round(df.describe().loc['mean'].values, 2))

    kFoldMeans.append(row)

# print(kFoldMeans)



kFoldMeansCols = list(df.describe().columns)

combinedCols.insert(0, "file")


# print(kFoldMeansCols)

totalMeans = pd.DataFrame(columns=combinedCols, data=kFoldMeans)

totalMeans = totalMeans.set_index('file')

cols = totalMeans.columns

cols = [c.replace("explanation", "expl").replace("accuracy", "acc").replace("complexity", "comp") for c in cols]

totalMeans.columns = cols

display(totalMeans)

print("Best explanations on minority class:\n")
for i in bestExplanationsDict:
    print(f"{i}:\n")
    print(bestExplanationsDict[i][1])



totalMeans.to_csv(f"./processingCache/totalMeans{date.today()}.csv")

,model_acc,extraction_time,f1,recall,precision,expl_acc_0,expl_fidelity_0,expl_comp_0,expl_acc_1,expl_fidelity_1,expl_comp_1
file,,,,,,,,,,,
breastCancer.csv,0.97,21.37,0.97,0.97,0.97,0.21,0.28,0.8,0.23,0.25,0.8
clusteredData.csv,0.92,26.41,0.51,0.52,0.50,0.50,0.93,2.4,0.34,0.58,1.4
expertLabelledData.csv,0.92,25.44,0.52,0.53,0.52,0.49,0.89,2.0,0.25,0.36,1.0
metricExtractedData.csv,0.92,25.57,0.54,0.55,0.54,0.50,0.89,2.2,0.35,0.56,1.8
staticData.csv,0.80,49.51,0.57,0.57,0.56,0.43,0.63,1.4,0.30,0.41,0.8


Best explanations on minority class:

breastCancer.csv:

[{'target_class': 0, 'explanation': '~Clump_Thickness_7 & ~Bland_Chromatin_10', 'explanation_accuracy': 0.5812562313060817, 'explanation_fidelity': 0.7357142857142858, 'explanation_complexity': 2}, {'target_class': 1, 'explanation': '~Clump_Thickness_1 & ~Bland_Chromatin_1', 'explanation_accuracy': 0.5736774193548386, 'explanation_fidelity': 0.5642857142857143, 'explanation_complexity': 2}]
clusteredData.csv:

[{'target_class': 0, 'explanation': 'Hemoglobin_Mean_high | ~Hemoglobin_Mean_high', 'explanation_accuracy': 0.4907161803713528, 'explanation_fidelity': 0.9895833333333334, 'explanation_complexity': 2}, {'target_class': 1, 'explanation': 'CVP_Mean_low & Platelets_Mean_high', 'explanation_accuracy': 0.47967479674796754, 'explanation_fidelity': 0.9479166666666666, 'explanation_complexity': 2}]
expertLabelledData.csv:

[{'target_class': 0, 'explanation': 'Hamoglobin_high | Hamoglobin_medium', 'explanation_accuracy': 0.489961139